# CoCoRaHS - Get station locations

Use this notebook to get the available stations and locations for a specified county between a specified date interval.
The output is a 

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import requests
import io
import pandas as pd

In [3]:
counties = ["AD","AL","BD","BN","BR","BU","CH","CR","CS","CP","CN","CL","CY","CT","CE","CK","CW","CM","DK","DW","DG","DP","ED","EW","EF","FY","FD","FK","FL","GL","GR","GY","HM","HN","HR","HD","HY","IR","JK","JS","JF","JR","JD","JH","KN","KK","KD","KX","LK","LS","LW","LE","LV","LG","MCD","MCH","MCL","MA","MP","MD","MN","MH","MS","MC","ME","MR","MO","MY","MG","MU","OG","PR","PY","PT","PK","PP","PL","PM","RH","RC","RI","SC","SL","SG","SY","ST","SH","SK","SP","TZ","UN","VR","WB","WR","WS","WY","WH","WD","WL","WM","WN","WF"]

In [4]:
params = {
   "ReportType": "Daily",
   "dtf": 1,
   "Format": "CSV",
   "State": "IL",
   "ReportDataType": "reportdate",
   "StartDate": "4/17/2013",
   "EndDate": "4/19/2013",
   "TimesInGMT": "False"
}
xml_responses = {}
for county in counties:
    params.update({
        "County": county
    })
    xml_responses[county] = requests.get("http://data.cocorahs.org/export/exportreports.aspx", params=params)

In [5]:
df = pd.DataFrame()
for county in counties:
    df_county = pd.read_csv(io.StringIO(xml_responses[county].text), delimiter=',',header=0)
    df = pd.concat([df,df_county])

In [6]:
df = df.drop_duplicates('StationNumber')

In [7]:
df_ = df[['StationNumber','StationName','Latitude','Longitude']]

In [8]:
df_.head()

,StationNumber,StationName,Latitude,Longitude
0,IL-AD-12,Quincy 3.7 SE,39.897127,-91.336417
0,IL-BN-5,Loves Park 3.7 ESE,42.325142,-88.939151
1,IL-BN-8,Poplar Grove 1.3 WSW,42.359990,-88.847010
2,IL-BN-9,Belvidere 1.6 NE,42.269941,-88.828382
3,IL-BN-14,Capron 0.1 N,42.399979,-88.742418


In [9]:
df_.to_csv('cocorahs_station_locations_{0}_{1}.csv'.format(params["StartDate"].replace('/',''), params["EndDate"].replace('/','')),index=False)